# Starbucks Capstone Challenge - Data Preparation

## Introduction

## Setup

In [1]:
import sys

!{sys.executable} -m pip install -e ../ --quiet

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sb_capstone.wrangling import (
    clean_portfolio,
    clean_profile,
    clean_transcript
)

from sb_capstone.shaping import (
    get_transcript_combined,
    get_transcript_group
)

In [3]:
portfolio = pd.read_csv('../data/processed/portfolio.csv')
portfolio = clean_portfolio(portfolio)
portfolio.head()

,id,offer_type,channels,reward,difficulty,duration
0,1,bogo,"[email, mobile, social]",10,10,7
1,2,bogo,"[web, email, mobile, social]",10,10,5
2,3,informational,"[web, email, mobile]",0,0,4
3,4,bogo,"[web, email, mobile]",5,5,7
4,5,discount,"[web, email]",5,20,10


In [4]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          10 non-null     int64   
 1   offer_type  10 non-null     category
 2   channels    10 non-null     object  
 3   reward      10 non-null     int64   
 4   difficulty  10 non-null     int64   
 5   duration    10 non-null     int64   
dtypes: category(1), int64(4), object(1)
memory usage: 742.0+ bytes


In [5]:
profile = pd.read_csv('../data/processed/profile.csv')
profile = clean_profile(profile)
profile.head()

,id,gender,age,income,became_member_on
0,1,NaN,NaN,NaN,2017-02-12
1,2,F,55.0,112000.0,2017-07-15
2,3,NaN,NaN,NaN,2018-07-12
3,4,F,75.0,100000.0,2017-05-09
4,5,NaN,NaN,NaN,2017-08-04


In [6]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                17000 non-null  int64         
 1   gender            14825 non-null  category      
 2   age               14825 non-null  float64       
 3   income            14825 non-null  float64       
 4   became_member_on  17000 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(2), int64(1)
memory usage: 548.2 KB


In [7]:
transcript = pd.read_csv('../data/processed/transcript.csv')
transcript = clean_transcript(transcript)
transcript.head()

,person_id,event,time,offer_id,amount,reward
0,4,offer_received,0,4,NaN,NaN
1,4,offer_viewed,6,4,NaN,NaN
2,4,transaction,132,0,19.89,NaN
3,4,offer_completed,132,4,NaN,5.0
4,4,transaction,144,0,17.78,NaN


In [8]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   person_id  306534 non-null  int64   
 1   event      306534 non-null  category
 2   time       306534 non-null  int64   
 3   offer_id   306534 non-null  category
 4   amount     138953 non-null  float64 
 5   reward     33579 non-null   float64 
dtypes: category(2), float64(2), int64(2)
memory usage: 9.9 MB


## Data Transformation

To help us compute the success of each offer, we need to simplify and compress the transcript by summarizing the timeline and summarizing them by extracting their offer group and offer type, because duplicate offers can occur on a single customer's timeline. If the customer has not received any offer, set the offer group and type to zero. 

Additional to that, we need to set if the offer was completed. The challenge for that is that informational offers does not have completion event, but as the presumption in the details of the data, that customers are influenced from the date of its validity. Therefore, we can set the completion event for informational offers if a transaction happens within its validity period.

In [9]:
transcript_all = transcript \
    .merge(portfolio, left_on="offer_id", right_on="id", how="left") \
    .rename(columns={"reward_x": "reward", "reward_y": "offer_reward"})

transcript_all.head()

,person_id,event,time,offer_id,amount,reward,id,offer_type,channels,offer_reward,difficulty,duration
0,4,offer_received,0,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
1,4,offer_viewed,6,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
2,4,transaction,132,0,19.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,offer_completed,132,4,NaN,5.0,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
4,4,transaction,144,0,17.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
transcript_all = get_transcript_combined(transcript_all)
transcript_all[transcript_all.person_id == 4]

,person_id,event,time,mapped_offer,amount,reward,offer_id,offer_type,channels,offer_reward,difficulty,duration,wave,day,offer_group
0,4,offer_received,0,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1,1,1.0
1,4,offer_viewed,6,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1,1,1.0
2,4,transaction,132,0,19.89,NaN,4.0,NaN,NaN,NaN,NaN,NaN,1,6,1.0
3,4,offer_completed,132,4,NaN,5.0,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1,6,1.0
4,4,transaction,144,0,17.78,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,7,-1.0
5,4,offer_received,168,8,NaN,NaN,8.0,informational,"[email, mobile, social]",0.0,0.0,3.0,2,8,2.0
6,4,offer_viewed,216,8,NaN,NaN,8.0,informational,"[email, mobile, social]",0.0,0.0,3.0,2,10,2.0
7,4,transaction,222,0,19.67,NaN,8.0,NaN,NaN,NaN,NaN,NaN,2,10,2.0
8,4,transaction,240,0,29.72,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2,11,-2.0
9,4,transaction,378,0,23.93,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3,16,-3.0


In [11]:
transcript_group = get_transcript_group(transcript_all, profile)
transcript_group[transcript_group.id == 4]

,id,wave,received,viewed,completed,amount,reward,non_offer_amount,mapped_offer,offer_type,...,web,email,mobile,social,gender,age,income,membership_year,membership_month,membership_day
3,4,2,True,True,True,19.67,0.0,29.72,8,informational,...,False,True,True,True,F,75.0,100000.0,2017,5,9
17003,4,3,False,False,False,0.00,0.0,23.93,0,no_offer,...,False,False,False,False,F,75.0,100000.0,2017,5,9
34003,4,4,True,True,True,0.00,10.0,0.00,1,bogo,...,False,True,True,True,F,75.0,100000.0,2017,5,9
51003,4,5,True,False,True,21.72,5.0,26.56,9,bogo,...,True,True,True,True,F,75.0,100000.0,2017,5,9
68003,4,6,False,False,False,0.00,0.0,0.00,0,no_offer,...,False,False,False,False,F,75.0,100000.0,2017,5,9
85003,4,1,True,True,True,19.89,5.0,17.78,4,bogo,...,True,True,True,False,F,75.0,100000.0,2017,5,9


## Save Dataset

Two dataset made available, one is summarized and the other one has timeline. We'll save them now for exploratory data analysis later.

In [12]:
transcript_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306534 entries, 0 to 306533
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   person_id     306534 non-null  int64   
 1   event         306534 non-null  category
 2   time          306534 non-null  int64   
 3   mapped_offer  306534 non-null  int64   
 4   amount        138953 non-null  float64 
 5   reward        33579 non-null   float64 
 6   offer_id      306534 non-null  float64 
 7   offer_type    167581 non-null  category
 8   channels      167581 non-null  object  
 9   offer_reward  167581 non-null  float64 
 10  difficulty    167581 non-null  float64 
 11  duration      167581 non-null  float64 
 12  wave          306534 non-null  category
 13  day           306534 non-null  category
 14  offer_group   306534 non-null  float64 
dtypes: category(4), float64(7), int64(3), object(1)
memory usage: 37.3+ MB


In [13]:
transcript_all.to_csv("../data/processed/transcript_all.csv", index=False)

In [14]:
transcript_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102000 entries, 0 to 101999
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   id                102000 non-null  int64   
 1   wave              102000 non-null  object  
 2   received          102000 non-null  bool    
 3   viewed            102000 non-null  bool    
 4   completed         102000 non-null  bool    
 5   amount            102000 non-null  float64 
 6   reward            102000 non-null  float64 
 7   non_offer_amount  102000 non-null  float64 
 8   mapped_offer      102000 non-null  int32   
 9   offer_type        102000 non-null  category
 10  difficulty        102000 non-null  float64 
 11  duration          102000 non-null  float64 
 12  web               102000 non-null  bool    
 13  email             102000 non-null  bool    
 14  mobile            102000 non-null  bool    
 15  social            102000 non-null  bool    
 16  ge

In [15]:
transcript_group.to_csv("../data/processed/transcript_group.csv", index=False)